In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, , , Waiting, )

In [2]:
df = spark.read.option("multiline", "true").json("Files/bing-fradulent-ecommerce-news.json")
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 82cb8e0b-6564-4cb2-9c94-b86ed0517a09)

In [3]:
df = spark.read.option("multiline", "true").json("Files/bing-fradulent-ecommerce-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-fradulent-ecommerce-news.json".
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 9495b872-be3b-4646-b2eb-acafdb4bb7d3)

In [4]:
df = df.select("value")

StatementMeta(, , , Waiting, )

In [5]:
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, cc2913f5-cfc8-4c8d-b3ff-543d585ba7e8)

In [6]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, , , Waiting, )

In [7]:
display(df_exploded)

StatementMeta(, , , Waiting, )

In [8]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, , , Waiting, )

In [9]:
print(json_list)

StatementMeta(, , , Waiting, )

In [10]:
import json
fraud_news_json=json.loads(json_list[7])

StatementMeta(, , , Waiting, )

In [11]:
print(fraud_news_json)

StatementMeta(, , , Waiting, )

In [ ]:
spark.table("binglakedb.tbl_latest_fraudnews").printSchema()


StatementMeta(, , , Waiting, )

In [12]:
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []

#process each JSON object from list
for json_str in json_list:
    try:
        #parse JSON string to dictionary
        article = json.loads(json_str)
        if article["json_object"].get("category") and article["json_object"].get("image",{}).get("thumbnail",{}).get("contentUrl"):
            #Extract information from dictionary
            title.append(article["json_object"]["name"])
            description.append(article["json_object"]["description"])
            category.append(article['json_object']['category'])
            url.append(article['json_object']['url'])
            image.append(article['json_object']['image']['thumbnail']['contentUrl'])
            provider.append(article['json_object']['provider'][0]['name'])
            datePublished.append(article['json_object']['datePublished'])
    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, , , Waiting, )

In [13]:
url

StatementMeta(, , , Waiting, )

**Converting the list to dataframe**

In [ ]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DateType


# Function to convert datetime string to date
def convert_to_date(date_str):
    return datetime.strptime(date_str.split("T")[0], "%Y-%m-%d").date()

# Convert datePublished to date format
datePublished = [convert_to_date(date) for date in datePublished]

# Combine the lists
data = list(zip(title, description, category, url, image, provider, datePublished))

# Define schema
schema = StructType(
    [
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("category", StringType(), True),
        StructField("url", StringType(), True),
        StructField("image", StringType(), True),
        StructField("provider", StringType(), True),
        StructField("datePublished", DateType(), True),
    ]
)

# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)


StatementMeta(, , , Waiting, )

In [14]:
from pyspark.sql.types import StructType, StructField, StringType, DateType
#combine the lists
data = list(zip(title,description,category,url,image,provider,datePublished))
#define Schema
schema = StructType(
    [
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("category", StringType(), True),
        StructField("url", StringType(), True),
        StructField("image", StringType(), True),
        StructField("provider", StringType(), True),
        StructField("datePublished", DateType(), True),
    ]
)
df_cleaned = spark.createDataFrame(data, schema=schema)

In [ ]:
spark.table("binglakedb.tbl_latest_fraudnews").printSchema()

StatementMeta(, , , Waiting, )

root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- datePublished: date (nullable = true)



**Writing the final Dataframe to the Lakehouse DB in a delta format**

In [15]:
display(df_cleaned)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, c08b56a8-de57-4aa8-a59c-9c81d45c2b36)

In [19]:
%%sql
SELECT count(*) FROM binglakedb.tbl_latest_fraudnews

StatementMeta(, , , Waiting, )

<Spark SQL result set with 1 rows and 1 fields>

In [20]:
from pyspark.sql.utils import AnalysisException

table_name = 'binglakedb.tbl_latest_fraudnews'
view_name = "vw_df_cleaned_final"

try:
    df_cleaned.write.format("delta").saveAsTable(table_name)
except AnalysisException as e:
    print("Table Already Exists")

    # Check if the temporary view exists and drop it if it does
    try:
        spark.catalog.dropTempView(view_name)
    except AnalysisException as e:
        print(f"Temporary view {view_name} does not exist or cannot be dropped: {e}")

    # Create the temporary view
    df_cleaned.createOrReplaceTempView(view_name)

    # Execute the MERGE statement
    try:
        spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING {view_name} AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR
            source_view.category <> target_table.category OR
            source_view.image <> target_table.image OR
            source_view.provider <> target_table.provider OR
            source_view.datePublished <> target_table.datePublished)
        THEN
        UPDATE SET
            target_table.title = source_view.title,
            target_table.description = source_view.description,
            target_table.category = source_view.category,
            target_table.image = source_view.image,
            target_table.provider = source_view.provider,
            target_table.datePublished = source_view.datePublished
        WHEN NOT MATCHED THEN
        INSERT (url, title, description, category, image, provider, datePublished)
        VALUES (source_view.url, source_view.title, source_view.description, source_view.category, source_view.image, source_view.provider, source_view.datePublished)
        """)
    except Exception as e:
        print(f"Error occurred during MERGE operation: {e}")

StatementMeta(, , , Waiting, )

Table Already Exists


In [ ]:
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 80532426-c68b-406e-ba65-ea7cc96a61f8)

In [21]:
%%sql
SELECT count(*) FROM binglakedb.tbl_latest_fraudnews

StatementMeta(, , , Waiting, )

<Spark SQL result set with 1 rows and 1 fields>